In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 축구화 관련

In [8]:
df_f = pd.read_csv('/content/drive/MyDrive/축구화 추천 프로젝트/data/football_boots_feature.csv')

df_f

,brand,silo,silo_kor,position1,position2,position3,feature
0,NIKE,phantom,팬텀,FW,MF,NaN,control
1,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
2,NIKE,tiempo,티엠포,MF,NaN,NaN,control
3,NIKE,premier,프리미어,NaN,NaN,NaN,power
4,adidas,F50,F50,FW,MF,NaN,speed
5,adidas,predator,프레데터,MF,FW,DF,control
6,adidas,X,엑스,FW,MF,NaN,speed
7,adidas,copa,코파,FW,MF,DF,control
8,mizuno,alpha,알파,FW,MF,NaN,speed
9,mizuno,morelia neo,모렐리아 네오,MF,FW,DF,speed


In [9]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_main_image(soup):
    img_tag = soup.select_one('.itemDetailPage-main-img img')
    if img_tag:
        img_url = img_tag.get('src')
        return f"https://www.crazy11.co.kr{img_url}" if img_url.startswith('/') else img_url
    return None

def extract_section_value(items, section_name):
    for item in items:
        if item.find('div', class_='mTC leftT') and item.find('div', class_='mTC leftT').text.strip() == section_name:
            values = item.find_all('div', class_='mcie on')
            return ", ".join([value.text.strip() for value in values]) if values else None
    return None

def extract_score(sections, score_name):
    for section in sections:
        if section.find('div', class_='bTC leftT') and section.find('div', class_='bTC leftT').text.strip() == score_name:
            inTB_elements = section.find_all('div', class_='inTB')
            for index, element in enumerate(inTB_elements, start=1):
                if 'on' in element.get('class', []):
                    return index
    return None

def extract_original_price(soup):
    price_span = soup.find("span", class_="itemDetailPage-price txtc-e4 txts-30")
    if price_span:
        original_price_span = price_span.find("span", class_="psale txts-18 txtc-a1")
        if original_price_span:
            return original_price_span.text.strip().replace(" 원", "").replace(",", "")
    return None

def extract_sale_price(soup):
    price_span = soup.find("span", class_="itemDetailPage-price txtc-e4 txts-30")
    if price_span:
        sale_price_text = price_span.contents[0].strip()
        return sale_price_text.replace(" 원", "").replace(",", "")
    return None

def extract_weight(soup):
    weight_sections = soup.find_all("span", class_="itembasic-bg")
    for section in weight_sections:
        title = section.find("span", class_="itembasic-tit")
        weight_text = section.find("span", class_="itembasic-txt")
        if title and weight_text and title.text.strip() == "무게":
            match = re.search(r'(\d+\.?\d*)g', weight_text.text.strip())
            if match:
                return match.group(1) + "g"
    return None

def crazy(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    response.encoding = 'cp949'
    soup = BeautifulSoup(response.text, "html.parser")

    title_div = soup.find("div", class_="itemDetailPage-main-tit txts-22 txts-fontB txtc-46")
    title = title_div.text.strip() if title_div else "Unknown Title"

    items = soup.find_all('div', class_='mTR')
    upper = extract_section_value(items, '갑피')
    ground = extract_section_value(items, '구장')

    sections = soup.find_all('div', class_='bTR')
    len_score = extract_score(sections, '길이')
    foot_score = extract_score(sections, '발볼')

    original_price = extract_original_price(soup)
    sale_price = extract_sale_price(soup)
    weight = extract_weight(soup)
    image_url = extract_main_image(soup)

    result = {
        'title': title,
        'original_price': original_price,
        'sale_price': sale_price,
        'upper': upper,
        'ground': ground,
        'len_score': len_score,
        'foot_score': foot_score,
        'weight': weight,
        'image_url': image_url,
        'url': url
    }
    return result

# 크롤링할 기본 URL
base_url = "https://www.crazy11.co.kr/shop/shopbrand.html"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
all_links = []

# 페이지 번호를 1부터 10까지 반복
for page in range(1, 11):
    params = {
        "type": "Y",
        "xcode": "257",
        "sort": "",
        "page": str(page),
    }
    response = requests.get(base_url, headers=headers, params=params)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    for a_tag in soup.select('a.itemInfo'):
        href = a_tag.get('href')
        if href and href.startswith('/shop/shopdetail.html'):
            full_url = "https://www.crazy11.co.kr" + href
            all_links.append(full_url)

data_list = []
for url in all_links:
    try:
        result = crazy(url)
        data_list.append(result)
    except Exception as e:
        print(f"Error: {url}: {e}")

df_b = pd.DataFrame(data_list)
df_b

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight,image_url,url
0,[입고예정2/17] 아디다스 프레데터 엘리트 FT FG (ID8966) 전용쌕 #,329000,329000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
1,푸마 퓨쳐 8 매치 크리에이티비티 FG/AG (10843101),129000,129000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
2,미즈노 알파 II 엘리트 MD (P1GA256209) #,229000,163000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
3,미즈노 알파 II 프로 MD (P1GA256409) #,179000,135000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
4,미즈노 알파 II 엘리트 AG (P1GA256309) #,229000,159000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
...,...,...,...,...,...,...,...,...,...,...
395,아디다스 X 크레이지패스트.1 LL FG (GY7381) #,319000,129000,"합성가죽, 니트",천연잔디,4.0,2.0,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
396,푸마 킹 얼티메이트 에너지 FG/AG (10755301) 전용쌕 #,269000,119000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
397,나이키 우먼스 팬텀 루나 엘리트 FG (FN8405-600) 전용쌕 #,329000,149000,None,None,NaN,NaN,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...
398,아디다스 X 크레이지패스트.1 FG (GY7417) #,309000,129000,"합성가죽, 니트",천연잔디,4.0,2.0,None,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...


### 데이터 전처리

In [10]:
# NaN 값 제거
if 'len_score' in df_b.columns:
    df_b = df_b.dropna(subset=['len_score'])

# upper 컬럼의 NaN 값을 '인조가죽 + 니트'로 채우기
df_b['upper'] = df_b['upper'].fillna('인조가죽 + 니트')

# 불필요한 컬럼 삭제
df_b = df_b.drop(columns=['Unnamed: 0'], errors='ignore')

# 인덱스 재설정
df_b = df_b.reset_index(drop=True)

# title 컬럼 전처리
df_b['title'] = df_b['title'].str.replace('신발끈|전용쌕|#|수입|/|신발주걱|인솔|UT 니트장갑 증정 EVENT|\.|,0', '', regex=True)
df_b['title'] = df_b['title'].str.replace(r'\[[^\]]+\] ', '', regex=True)
df_b["title"] = df_b["title"].str.replace(r"\s*\(.*?\)", "", regex=True)

# title, original_price 중복 제거
df_b = df_b.drop_duplicates(subset=["title", "original_price"], keep="first")

# 중복 확인
print("총 데이터 개수:", len(df_b))
print("title 컬럼 내 유니크 값 개수:", df_b['title'].nunique())
print("중복된 행 개수:", df_b.duplicated(subset=['title', 'original_price']).sum())

# 특정 키워드 포함된 행 삭제
df_b = df_b[~df_b["title"].str.contains("트루삭스", na=False)]
df_b = df_b[~df_b["title"].str.contains("우먼스", na=False)]

# weight 컬럼에서 숫자만 추출하고 컬럼명 변경
df_b.rename(columns={'weight': 'weight(g)'}, inplace=True)
df_b['weight(g)'] = df_b['weight(g)'].astype(str).str.extract(r'(\d+)')

총 데이터 개수: 164
title 컬럼 내 유니크 값 개수: 155
중복된 행 개수: 0


<ipython-input-10-57e7a4f55829>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b['upper'] = df_b['upper'].fillna('인조가죽 + 니트')


In [11]:
# 조건에 따라 'silo' 값 설정
def extract_silo(title):
    split_title = title.split(' ')  # 공백 기준 분리
    if ('더' in split_title or '줌' in split_title):
        return split_title[2]  # 두번째 단어 반환
    elif ('킹' in split_title):
        return ' '.join(split_title[1:3])  # 두번째와 세번째 단어를 합쳐 반환
    elif (split_title[1] == '모렐리아' and split_title[2] == '네오'):
        return ' '.join(split_title[1:3])
    return split_title[1]  # 첫번째 단어만 반환

# 'silo' 컬럼 생성
df_b['silo'] = df_b['title'].apply(extract_silo)

In [12]:
df_b['silo'].unique()

array(['프레데터', 'F50', '퓨처', '울트라', '머큐리얼', '팬텀', '티엠포', '모렐리아', '코파',
       '킹 얼티메이트', '모렐리아 네오', '모나르시다', '알파', 'X', '프리미어'], dtype=object)

In [13]:
df_b['title'].nunique()

154

- 중복된 값들 확인

In [14]:
# 중복된 값 추출
duplicates = df_b['title'][df_b['title'].duplicated()]
print(duplicates)

6        아디다스 프레데터 엘리트 LL FG  
88          아디다스 프레데터 엘리트 FG  
90           아디다스 F50 엘리트 FG  
91           아디다스 F50 리그 FGMG 
168           푸마 킹 얼티메이트 FGAG 
177     아디다스 프레데터 엘리트 AG 2G3G 
178       아디다스 코파 퓨어 2 엘리트 FG 
205    나이키 팬텀 GX II 아카데미 FGMG 
214             푸마 킹 얼티메이트 MG 
Name: title, dtype: object


In [15]:
filtered_df = df_b[df_b['title'].str.contains('모렐리아', na=False)]
filtered_df

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight(g),image_url,url,silo
13,미즈노 모렐리아 II JP,279000,225000,"캥거루, 인조가죽","천연잔디, 인조잔디, 맨땅",2.0,3.0,202,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아
14,미즈노 모렐리아 II 프로 MD,179000,129000,"캥거루, 인조가죽","천연잔디, 인조잔디",2.0,2.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아
15,미즈노 모렐리아 II 클럽 MD,99000,75000,인조가죽,"천연잔디, 인조잔디",3.0,1.0,208,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아
32,미즈노 모렐리아 네오 IV 프로 AG,169000,134000,"캥거루, 인조가죽","천연잔디, 인조잔디",2.0,2.0,221,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오
44,미즈노 모렐리아 II 클럽 MD,99000,77000,인조가죽,"천연잔디, 인조잔디",3.0,1.0,208,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아
45,미즈노 모렐리아 네오 IV 프로 MD,169000,135000,"캥거루, 인조가죽","천연잔디, 인조잔디, 맨땅",2.0,2.0,224,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오
46,미즈노 모렐리아 네오 IV 엘리트 MD,239000,185000,"캥거루, 인조가죽","천연잔디, 인조잔디, 맨땅",3.0,2.0,189,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오
47,미즈노 모렐리아 네오 IV 베타 엘리트 MD,269000,215000,"캥거루, 인조가죽, 합성가죽","천연잔디, 인조잔디, 맨땅",3.0,2.0,186,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오
48,미즈노 모렐리아 네오 IV JP MD,309000,254000,"캥거루, 인조가죽","천연잔디, 인조잔디, 맨땅",2.0,3.0,192,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오
49,미즈노 모렐리아 네오 IV 베타 MD,339000,275000,"캥거루, 인조가죽, 합성가죽","천연잔디, 인조잔디, 맨땅",2.0,2.0,190,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,모렐리아 네오


In [16]:
df_b[df_b['title'].isin(duplicates)].sort_values(by = 'title')

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight(g),image_url,url,silo
36,나이키 팬텀 GX II 아카데미 FGMG,109000,81000,합성가죽,"천연잔디, 인조잔디, 맨땅",4.0,2.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,팬텀
205,나이키 팬텀 GX II 아카데미 FGMG,99000,85000,합성가죽,"천연잔디, 인조잔디, 맨땅",4.0,2.0,220,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,팬텀
20,아디다스 F50 리그 FGMG,109000,71000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,209,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,F50
91,아디다스 F50 리그 FGMG,119000,77000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,209,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,F50
3,아디다스 F50 엘리트 FG,319000,235000,"인조가죽, 합성가죽",천연잔디,3.0,3.0,163,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,F50
90,아디다스 F50 엘리트 FG,299000,209000,"인조가죽, 합성가죽",천연잔디,3.0,3.0,163,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,F50
178,아디다스 코파 퓨어 2 엘리트 FG,289000,169000,인조가죽,천연잔디,4.0,2.0,213,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,코파
86,아디다스 코파 퓨어 2 엘리트 FG,259000,169000,인조가죽,천연잔디,4.0,2.0,213,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,코파
83,아디다스 프레데터 엘리트 AG 2G3G,289000,209000,합성가죽,인조잔디,4.0,3.0,256,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,프레데터
177,아디다스 프레데터 엘리트 AG 2G3G,309000,179000,합성가죽,인조잔디,4.0,3.0,256,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,프레데터


In [17]:
# 중복 없음 확인 중복된 title 중에서 price가 가장 큰 값 남기기
df_b = df_b.loc[df_b.groupby('title')['original_price'].idxmax()]

# 중복 없음 확인
duplicates = df_b['title'][df_b['title'].duplicated()]
print(duplicates)

Series([], Name: title, dtype: object)


In [18]:
df_f.loc[df_f['silo_kor'] == '엑스', 'silo_kor'] = 'X'

df_f

,brand,silo,silo_kor,position1,position2,position3,feature
0,NIKE,phantom,팬텀,FW,MF,NaN,control
1,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
2,NIKE,tiempo,티엠포,MF,NaN,NaN,control
3,NIKE,premier,프리미어,NaN,NaN,NaN,power
4,adidas,F50,F50,FW,MF,NaN,speed
5,adidas,predator,프레데터,MF,FW,DF,control
6,adidas,X,X,FW,MF,NaN,speed
7,adidas,copa,코파,FW,MF,DF,control
8,mizuno,alpha,알파,FW,MF,NaN,speed
9,mizuno,morelia neo,모렐리아 네오,MF,FW,DF,speed


In [19]:
# 조인 실행
result = pd.merge(df_b, df_f, left_on='silo', right_on='silo_kor', how='left')

result.head()

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight(g),image_url,url,silo_x,brand,silo_y,silo_kor,position1,position2,position3,feature
0,나이키 더 프리미어 III FG,129000,83000,"소가죽, 합성가죽",천연잔디,3.0,2.0,251,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,프리미어,NIKE,premier,프리미어,NaN,NaN,NaN,power
1,나이키 줌 머큐리얼 베이퍼 15 MDS 아카데미 FGMG,109000,69000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
2,나이키 줌 머큐리얼 베이퍼 15 아카데미 FGMG,109000,89000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,222,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
3,나이키 줌 머큐리얼 베이퍼 16 MDS 아카데미 FGMG,119000,91000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
4,나이키 줌 머큐리얼 베이퍼 16 아카데미 FGMG,109000,83000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed


In [20]:
result

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight(g),image_url,url,silo_x,brand,silo_y,silo_kor,position1,position2,position3,feature
0,나이키 더 프리미어 III FG,129000,83000,"소가죽, 합성가죽",천연잔디,3.0,2.0,251,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,프리미어,NIKE,premier,프리미어,NaN,NaN,NaN,power
1,나이키 줌 머큐리얼 베이퍼 15 MDS 아카데미 FGMG,109000,69000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
2,나이키 줌 머큐리얼 베이퍼 15 아카데미 FGMG,109000,89000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,222,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
3,나이키 줌 머큐리얼 베이퍼 16 MDS 아카데미 FGMG,119000,91000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
4,나이키 줌 머큐리얼 베이퍼 16 아카데미 FGMG,109000,83000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,푸마 퓨처 얼티메이트 FGAG,289000,119000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,217,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
150,푸마 퓨처 얼티메이트 MG,289000,159000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디, 맨땅",5.0,3.0,219,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
151,푸마 퓨처 얼티메이트 로우 FGAG,279000,139000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,211,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
152,푸마 퓨처 얼티메이트 에너지 FGAG,299000,149000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable


In [21]:
result.to_csv('/content/drive/MyDrive/축구화 추천 프로젝트/boots.csv', index=False)

In [22]:
df = pd.read_csv('/content/drive/MyDrive/축구화 추천 프로젝트/boots.csv')

df

,title,original_price,sale_price,upper,ground,len_score,foot_score,weight(g),image_url,url,silo_x,brand,silo_y,silo_kor,position1,position2,position3,feature
0,나이키 더 프리미어 III FG,129000,83000,"소가죽, 합성가죽",천연잔디,3.0,2.0,251.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,프리미어,NIKE,premier,프리미어,NaN,NaN,NaN,power
1,나이키 줌 머큐리얼 베이퍼 15 MDS 아카데미 FGMG,109000,69000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
2,나이키 줌 머큐리얼 베이퍼 15 아카데미 FGMG,109000,89000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,222.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
3,나이키 줌 머큐리얼 베이퍼 16 MDS 아카데미 FGMG,119000,91000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
4,나이키 줌 머큐리얼 베이퍼 16 아카데미 FGMG,109000,83000,합성가죽,"천연잔디, 인조잔디, 맨땅",3.0,2.0,195.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,머큐리얼,NIKE,mercurial,머큐리얼,FW,MF,NaN,speed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,푸마 퓨처 얼티메이트 FGAG,289000,119000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,217.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
150,푸마 퓨처 얼티메이트 MG,289000,159000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디, 맨땅",5.0,3.0,219.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
151,푸마 퓨처 얼티메이트 로우 FGAG,279000,139000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,211.0,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable
152,푸마 퓨처 얼티메이트 에너지 FGAG,299000,149000,"인조가죽, 합성가죽, 니트","천연잔디, 인조잔디",5.0,3.0,NaN,https://www.crazy11.co.kr/shopimages/winv007/0...,https://www.crazy11.co.kr/shop/shopdetail.html...,퓨처,PUMA,future,퓨처,FW,MF,DF,comfortable


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           154 non-null    object 
 1   original_price  154 non-null    int64  
 2   sale_price      154 non-null    int64  
 3   upper           154 non-null    object 
 4   ground          154 non-null    object 
 5   len_score       154 non-null    float64
 6   foot_score      154 non-null    float64
 7   weight(g)       134 non-null    float64
 8   image_url       154 non-null    object 
 9   url             154 non-null    object 
 10  silo_x          154 non-null    object 
 11  brand           154 non-null    object 
 12  silo_y          154 non-null    object 
 13  silo_kor        154 non-null    object 
 14  position1       147 non-null    object 
 15  position2       132 non-null    object 
 16  position3       61 non-null     object 
 17  feature         154 non-null    obj